In [ ]:
# 1. Instalar librerías necesarias
# (Instalamos accelerate y transformers para que todo funcione fluido en la T4)
!pip install transformers datasets accelerate

# 2. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# --- CONFIGURACIÓN DE RUTAS EXACTAS (Según tu imagen) ---

# Nota: He añadido "ASCII-GPT/" a la ruta porque los archivos están dentro de esa carpeta.
path_zip_drive = '/content/drive/MyDrive/ASCII-GPT/modelo_travian_v2_completo.zip'
path_dataset_drive = '/content/drive/MyDrive/ASCII-GPT/dataset_TRAVIAN_CLEAN_V2.txt'

# Directorios de trabajo temporal en Colab (Aquí descomprimiremos)
extract_path = '/content/modelo_v2'
output_dir = '/content/modelo_travian_v3_refinado'

# --- VERIFICACIÓN Y DESCOMPRESIÓN ---

# Verificamos si los archivos existen antes de intentar nada para evitar errores feos
if os.path.exists(path_zip_drive):
    print(f"✅ Archivo ZIP encontrado: {path_zip_drive}")

    # Descomprimir el modelo v2
    if not os.path.exists(extract_path):
        print("⏳ Descomprimiendo modelo v2...")
        with zipfile.ZipFile(path_zip_drive, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("🚀 ¡Modelo descomprimido y listo en /content/modelo_v2!")
    else:
        print("ℹ️ El modelo ya estaba descomprimido en la sesión.")
else:
    print(f"❌ ERROR: No encuentro el archivo en: {path_zip_drive}")
    print("Asegúrate de que el nombre del archivo en Drive sea EXACTAMENTE 'modelo_travian_v2_completo.zip'")

if os.path.exists(path_dataset_drive):
    print(f"✅ Dataset encontrado: {path_dataset_drive}")
else:
    print(f"❌ ERROR: No encuentro el dataset en: {path_dataset_drive}")
    print("Verifica si tiene extensión .txt al final o si Google Drive se la ocultó.")

Mounted at /content/drive
✅ Archivo ZIP encontrado: /content/drive/MyDrive/ASCII-GPT/modelo_travian_v2_completo.zip
⏳ Descomprimiendo modelo v2...
🚀 ¡Modelo descomprimido y listo en /content/modelo_v2!
✅ Dataset encontrado: /content/drive/MyDrive/ASCII-GPT/dataset_TRAVIAN_CLEAN_V2.txt


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Cargar el Tokenizer y el Modelo desde la carpeta descomprimida
model_path = extract_path # Apunta a donde descomprimimos

print(f"Cargando modelo desde: {model_path}")
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Asegurar token de pad (GPT-2 no lo tiene por defecto)
tokenizer.pad_token = tokenizer.eos_token

print("✅ Modelo v2 cargado. Loss anterior reportado: 0.61")
print("Listo para entrenamiento incremental.")

Cargando modelo desde: /content/modelo_v2
✅ Modelo v2 cargado. Loss anterior reportado: 0.61
Listo para entrenamiento incremental.


In [ ]:
# Función para cargar dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
        overwrite_cache=True
    )

print("Procesando Dataset...")
# Usamos el dataset de tu Drive
train_dataset = load_dataset(path_dataset_drive, tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)
print("✅ Dataset tokenizado y listo.")

Procesando Dataset...


/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


✅ Dataset tokenizado y listo.


In [ ]:
import os

# --- DESACTIVAR WANDB (Para que no pida contraseña) ---
os.environ["WANDB_DISABLED"] = "true"

from transformers import TrainingArguments, Trainer

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=2,            # Mantenemos pocas épocas
    per_device_train_batch_size=4, # Batch size bajo para T4
    save_steps=500,
    save_total_limit=2,
    learning_rate=1e-5,            # Learning Rate bajo para refinamiento
    prediction_loss_only=True,
    logging_steps=50,
    fp16=True,                     # Mixed Precision
    report_to="none"               # <--- ESTO EVITA QUE PIDA WANDB
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("🚀 Iniciando Fase 3: Refinamiento Estructural (Sin distracciones)...")
trainer.train()
print("🏁 Entrenamiento finalizado.")

🚀 Iniciando Fase 3: Refinamiento Estructural (Sin distracciones)...


Step,Training Loss
50,1.174400
100,1.121200
150,1.210500
200,1.182000
250,1.096100
300,1.128800
350,1.098100
400,1.136800
450,1.143700
500,1.099700


🏁 Entrenamiento finalizado.


In [ ]:
import shutil
import os

# 1. Guardar el modelo crudo en el disco local de Colab
model_save_path = "/content/modelo_travian_v3_final"
print("💾 Guardando archivos del modelo...")
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

# 2. Comprimir en un ZIP
print("📦 Comprimiendo...")
shutil.make_archive("/content/modelo_travian_v3_final", 'zip', model_save_path)

# 3. Mover a tu carpeta ASCII-GPT en Drive
# Ruta destino exacta
destination_path = "/content/drive/MyDrive/ASCII-GPT/modelo_travian_v3_final.zip"

print(f"🚀 Subiendo a Drive: {destination_path} ...")
shutil.move("/content/modelo_travian_v3_final.zip", destination_path)

print("✅ ¡BACKUP COMPLETADO! Ya puedes respirar tranquilo.")

💾 Guardando archivos del modelo...
📦 Comprimiendo...
🚀 Subiendo a Drive: /content/drive/MyDrive/ASCII-GPT/modelo_travian_v3_final.zip ...
✅ ¡BACKUP COMPLETADO! Ya puedes respirar tranquilo.


In [ ]:
def generate_ascii_rigid(prompt_text, max_length=500):
    input_ids = tokenizer.encode(prompt_text, return_tensors='pt').to(model.device)

    # Configuración "Delineante Técnico"
    sample_outputs = model.generate(
        input_ids,
        max_length=max_length,
        do_sample=False,        # <--- GREEDY DECODING (Elegir siempre lo más probable)
        # temperature=0.2,      # No se usa si do_sample=False, pero si lo activas, úsalo bajo.
        repetition_penalty=1.2, # Castigar repeticiones infinitas
        pad_token_id=tokenizer.eos_token_id
    )

    print(f"--- Generación para: '{prompt_text}' ---")
    print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))
    print("-" * 40)

# PRUEBAS
# Prueba con prompts que sepas que existen en el dataset
generate_ascii_rigid("Buildings Castle")
generate_ascii_rigid("Map Forest")
generate_ascii_rigid("Travian Village")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


--- Generación para: 'Buildings Castle' ---
Buildings Castle
AI:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
----------------------------------------
--- Generación para: 'Map Forest' ---
Map Forest 
AI:
          .-""-.                                                                                                                                                                                                                                                                                                                                          

In [ ]:
# --- CELDA 6: PRUEBA DE INFERENCIA (Modo Arquitecto) ---
import torch

def generate_ascii_rigid(prompt_text, max_length=500):
    # Asegurarnos de que el modelo esté en modo evaluación
    model.eval()

    # Preparar el input
    input_ids = tokenizer.encode(prompt_text, return_tensors='pt').to(model.device)

    # Generación Deterministica (Greedy Decoding)
    # Buscamos la máxima coherencia estructural, sacrificando creatividad.
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            do_sample=False,        # <--- ESTO ES CLAVE: Apagamos la aleatoriedad
            repetition_penalty=1.2, # Castigo suave a la repetición para evitar bucles
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decodificar y mostrar
    print(f"\n🏗️ GENERANDO ARQUITECTURA PARA: '{prompt_text}'")
    print("=" * 60)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(generated_text)
    print("=" * 60)

# --- ZONA DE PRUEBAS ---
# Probamos los conceptos clave de Travian
generate_ascii_rigid("Buildings Castle")
generate_ascii_rigid("Map Forest")
generate_ascii_rigid("Travian Village")


🏗️ GENERANDO ARQUITECTURA PARA: 'Buildings Castle'
Buildings Castle
AI:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

🏗️ GENERANDO ARQUITECTURA PARA: 'Map Forest'
Map Forest
AI:
                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
# --- INFERENCIA AVANZADA v3.1 (Desbloqueo de Creatividad) ---
import torch

def generate_ascii_creative(prompt_text, temperature=0.6):
    model.eval()

    # TRUCO: Añadimos un salto de línea (\n) para decirle "empieza a dibujar YA"
    # Si el dataset tenía etiquetas, a veces ayuda simularlas.
    # Probamos con el prompt directo + salto de línea.
    full_prompt = f"{prompt_text}\n"

    input_ids = tokenizer.encode(full_prompt, return_tensors='pt').to(model.device)

    print(f"🎨 Intentando dibujar: '{prompt_text}' con Temp {temperature}...")
    print("=" * 60)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=600,         # Damos espacio suficiente
            do_sample=True,         # <--- ACTIVAMOS CREATIVIDAD (Muestreo)
            top_k=50,               # Elegir entre los 50 mejores tokens (evita basura pura)
            top_p=0.95,             # Nucleus sampling
            temperature=temperature,# Control de "locura"
            repetition_penalty=1.1, # Penalización baja (necesitamos repetir caracteres para paredes |||)
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(generated_text)
    print("=" * 60 + "\n")

# --- RONDA DE PRUEBAS ---
# Probamos con diferentes temperaturas para ver cuál "despierta" al modelo

# 1. Intento equilibrado
generate_ascii_creative("Buildings Castle", temperature=0.6)

# 2. Intento más creativo (si el anterior sale blanco)
generate_ascii_creative("Map Forest", temperature=0.8)

# 3. Intento específico de Travian
generate_ascii_creative("Travian Village", temperature=0.5)

🎨 Intentando dibujar: 'Buildings Castle' con Temp 0.6...
Buildings Castle
AI:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

🎨 Intentando dibujar: 'Map Forest' con Temp 0.8...
Map Forest
AI:
                  _,---._                                         
        .'                        `.                                      
         |              ,-`-.                   /       \                                       
         |    :       

In [ ]:
# --- SCRIPT: THE GREAT ASCII HEIST (Scraper de asciiart.eu) ---
import requests
from bs4 import BeautifulSoup
import time
import random

# Las categorías más jugosas (puedes añadir más si ves la web)
URLS_CATEGORIAS = [
    "https://www.asciiart.eu/animals/cats",
    "https://www.asciiart.eu/animals/dogs",
    "https://www.asciiart.eu/animals/bats",
    "https://www.asciiart.eu/animals/birds-land",
    "https://www.asciiart.eu/animals/insects",
    "https://www.asciiart.eu/animals/reptiles",
    "https://www.asciiart.eu/space/aliens",
    "https://www.asciiart.eu/space/planets",
    "https://www.asciiart.eu/video-games/mario",
    "https://www.asciiart.eu/video-games/sonic",
    "https://www.asciiart.eu/movies/star-wars",
    "https://www.asciiart.eu/computers/computers",
    "https://www.asciiart.eu/food-and-drinks/coffee",
    "https://www.asciiart.eu/objects/cars",
    "https://www.asciiart.eu/music/musical-instruments",
    "https://www.asciiart.eu/people/skeletons",
    "https://www.asciiart.eu/weapons/swords",
]

OUTPUT_FILE = "dataset_ASCII_GENERAL_HUMANO.txt"
MIN_CHARS = 30  # Ignorar dibujos muy enanos (ruido)
MAX_CHARS = 2000 # Ignorar dibujos gigantes que rompen el contexto de GPT-2

def get_ascii_from_url(url, label_category):
    print(f"🕷️ Scrapeando categoría: {label_category}...")
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')

        # En asciiart.eu, el arte está dentro de etiquetas <pre>
        art_blocks = soup.find_all('pre')

        collected = 0
        entries = []

        for art in art_blocks:
            text = art.get_text()
            if MIN_CHARS < len(text) < MAX_CHARS:
                # Limpieza básica
                text = text.replace("\r", "")

                # FORMATO PROMPT:
                # Input: [Category]
                # AI: [Art]
                entry = f"{label_category}\nAI:\n{text}\n<|endoftext|>\n"
                entries.append(entry)
                collected += 1

        print(f"   ✅ Encontrados {collected} dibujos.")
        return entries

    except Exception as e:
        print(f"   ❌ Error en {url}: {e}")
        return []

# --- EJECUCIÓN ---
print("🚀 Iniciando recolección de Arte Humano...")
total_entries = 0

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for url in URLS_CATEGORIAS:
        # Extraer nombre bonito de la URL (ej: "cats")
        category_name = url.split("/")[-1].replace("-", " ").title()

        entries = get_ascii_from_url(url, category_name)

        for entry in entries:
            f.write(entry)

        total_entries += len(entries)
        time.sleep(1) # Ser educados con el servidor

print("="*50)
print(f"🎉 ¡RECOLECCIÓN COMPLETADA! {total_entries} obras maestras guardadas.")
print(f"📁 Archivo listo: {OUTPUT_FILE}")

🚀 Iniciando recolección de Arte Humano...
🕷️ Scrapeando categoría: Cats...
   ✅ Encontrados 81 dibujos.
🕷️ Scrapeando categoría: Dogs...
   ✅ Encontrados 55 dibujos.
🕷️ Scrapeando categoría: Bats...
   ✅ Encontrados 18 dibujos.
🕷️ Scrapeando categoría: Birds Land...
   ✅ Encontrados 67 dibujos.
🕷️ Scrapeando categoría: Insects...
   ✅ Encontrados 1 dibujos.
🕷️ Scrapeando categoría: Reptiles...
   ✅ Encontrados 1 dibujos.
🕷️ Scrapeando categoría: Aliens...
   ✅ Encontrados 36 dibujos.
🕷️ Scrapeando categoría: Planets...
   ✅ Encontrados 11 dibujos.
🕷️ Scrapeando categoría: Mario...
   ✅ Encontrados 1 dibujos.
🕷️ Scrapeando categoría: Sonic...
   ✅ Encontrados 1 dibujos.
🕷️ Scrapeando categoría: Star Wars...
   ✅ Encontrados 12 dibujos.
🕷️ Scrapeando categoría: Computers...
   ✅ Encontrados 36 dibujos.
🕷️ Scrapeando categoría: Coffee...
   ✅ Encontrados 1 dibujos.
🕷️ Scrapeando categoría: Cars...
   ✅ Encontrados 1 dibujos.
🕷️ Scrapeando categoría: Musical Instruments...
   ✅ Encontrados

In [ ]:
# --- SCRIPT: THE ULTIMATE CRAWLER (Aspiradora de asciiart.eu) ---
import requests
from bs4 import BeautifulSoup
import time

# Lista basada en tu captura de pantalla (Categorías Padre)
# El script entrará aquí y buscará las subcategorías automáticamente.
MAIN_CATEGORIES = [
    "animals",
    "art-and-design",
    "books",
    "buildings-and-places",
    "cartoons",
    "clothing-and-accessories",
    "comics",
    "computers",
    "electronics",
    "food-and-drinks",
    "holiday-and-events",
    "logos",
    "miscellaneous",
    "movies",
    "music",
    "mythology",
    "nature",
    "people",
    "plants",
    "religion",
    "space",
    "sports-and-outdoors",
    "television",
    "toys",
    "vehicles",
    "video-games",
    "weapons"
]

BASE_URL = "https://www.asciiart.eu"
OUTPUT_FILE = "dataset_ASCII_MASSIVE.txt"
MIN_CHARS = 20   # Filtrar ruido
MAX_CHARS = 2500 # Filtrar cosas demasiado grandes

def get_soup(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        return BeautifulSoup(response.text, 'html.parser')
    except:
        return None

# --- FASE 1: MAPEO DE SUBCATEGORÍAS ---
print("🗺️  Iniciando mapeo del sitio...")
all_urls_to_scrape = []

for category in MAIN_CATEGORIES:
    cat_url = f"{BASE_URL}/{category}"
    print(f"   📂 Explorando índice: {category.upper()}...")

    soup = get_soup(cat_url)
    if soup:
        # En asciiart.eu, los enlaces a subcategorías están en una lista "directory-columns"
        # Ojo: Buscamos enlaces que contengan la categoría en su href
        links = soup.find_all('a', href=True)

        found_subs = 0
        for link in links:
            href = link['href']
            # Validar que es una subcategoría válida y no un enlace externo o basura
            if href.startswith(f"/{category}/") and len(href.split('/')) > 2:
                full_url = BASE_URL + href
                if full_url not in all_urls_to_scrape:
                    all_urls_to_scrape.append(full_url)
                    found_subs += 1

        print(f"      -> Encontradas {found_subs} subcategorías (ej: {category}/xyz).")
    time.sleep(0.5)

print(f"\n📋 TOTAL DE PÁGINAS A PROCESAR: {len(all_urls_to_scrape)}")
print("="*40)

# --- FASE 2: EXTRACCIÓN MASIVA ---
print("🚜 Iniciando extracción de arte...")

total_drawings = 0
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for i, url in enumerate(all_urls_to_scrape):
        # Crear etiqueta bonita: "animals/bison" -> "Animals Bison"
        label_parts = url.replace(BASE_URL + "/", "").split("/")
        prompt_label = " ".join([p.replace("-", " ").title() for p in label_parts])

        # Feedback visual cada 10 URLs
        if i % 10 == 0:
            print(f"   [{i}/{len(all_urls_to_scrape)}] Procesando: {prompt_label}...")

        soup = get_soup(url)
        if soup:
            art_blocks = soup.find_all('pre')
            count = 0
            for art in art_blocks:
                text = art.get_text()
                # Filtros de calidad
                if MIN_CHARS < len(text) < MAX_CHARS:
                    # Limpieza
                    text = text.replace("\r", "")

                    # FORMATO FINAL
                    entry = f"{prompt_label}\nAI:\n{text}\n<|endoftext|>\n"
                    f.write(entry)
                    count += 1
            total_drawings += count

        # Pausa pequeña para no bloquear la IP
        time.sleep(0.2)

print("="*50)
print(f"🎉 ¡GRAN ROBO COMPLETADO! Se han robado {total_drawings} dibujos ASCII.")
print(f"💾 Archivo guardado: {OUTPUT_FILE}")

🗺️  Iniciando mapeo del sitio...
   📂 Explorando índice: ANIMALS...
      -> Encontradas 31 subcategorías (ej: animals/xyz).
   📂 Explorando índice: ART-AND-DESIGN...
      -> Encontradas 18 subcategorías (ej: art-and-design/xyz).
   📂 Explorando índice: BOOKS...
      -> Encontradas 8 subcategorías (ej: books/xyz).
   📂 Explorando índice: BUILDINGS-AND-PLACES...
      -> Encontradas 16 subcategorías (ej: buildings-and-places/xyz).
   📂 Explorando índice: CARTOONS...
      -> Encontradas 23 subcategorías (ej: cartoons/xyz).
   📂 Explorando índice: CLOTHING-AND-ACCESSORIES...
      -> Encontradas 16 subcategorías (ej: clothing-and-accessories/xyz).
   📂 Explorando índice: COMICS...
      -> Encontradas 17 subcategorías (ej: comics/xyz).
   📂 Explorando índice: COMPUTERS...
      -> Encontradas 15 subcategorías (ej: computers/xyz).
   📂 Explorando índice: ELECTRONICS...
      -> Encontradas 12 subcategorías (ej: electronics/xyz).
   📂 Explorando índice: FOOD-AND-DRINKS...
      -> Encont

In [ ]:
# --- FUSIONADOR FINAL (Solo Calidad Humana) ---
import os

# Archivos a unir
files_to_merge = [
    # El nuevo gigante que acabamos de scrapear
    "dataset_ASCII_MASSIVE.txt",

    # Tu dataset limpio anterior (si lo tienes a mano, suma variedad)
    # Asegúrate de que la ruta sea correcta, si no existe, el script lo ignorará sin romper nada.
    "/content/drive/MyDrive/ASCII-GPT/dataset_TRAVIAN_CLEAN_V2.txt"
]

output_filename = "dataset_FINAL_V3.txt"
total_lines = 0

print(f"🔥 Iniciando Fusión Nuclear de Datasets...")

with open(output_filename, 'w', encoding='utf-8') as outfile:
    for fname in files_to_merge:
        if os.path.exists(fname):
            print(f"   ➕ Añadiendo contenido de: {fname}")
            try:
                with open(fname, 'r', encoding='utf-8', errors='ignore') as infile:
                    content = infile.read()
                    outfile.write(content)
                    outfile.write("\n<|endoftext|>\n") # Asegurar separación limpia
                    total_lines += content.count('\n')
            except Exception as e:
                print(f"      ❌ Error leyendo {fname}: {e}")
        else:
            print(f"   ⚠️ Archivo no encontrado (saltando): {fname}")

# Verificamos el tamaño final
file_size_mb = os.path.getsize(output_filename) / (1024 * 1024)
print("="*50)
print(f"✅ ¡DATASET MAESTRO CREADO!: {output_filename}")
print(f"📊 Tamaño: {file_size_mb:.2f} MB")
print(f"📝 Líneas aproximadas: {total_lines}")
print("¡Estás listo para entrenar el modelo definitivo!")

🔥 Iniciando Fusión Nuclear de Datasets...
   ➕ Añadiendo contenido de: dataset_ASCII_MASSIVE.txt
   ➕ Añadiendo contenido de: /content/drive/MyDrive/ASCII-GPT/dataset_TRAVIAN_CLEAN_V2.txt
✅ ¡DATASET MAESTRO CREADO!: dataset_FINAL_V3.txt
📊 Tamaño: 7.30 MB
📝 Líneas aproximadas: 191981
¡Estás listo para entrenar el modelo definitivo!


In [ ]:
import shutil
import os

source = "dataset_FINAL_V3.txt"
destination = "/content/drive/MyDrive/ASCII-GPT/dataset_FINAL_V3.txt"

if os.path.exists(source):
    print(f"🚑 Rescatando {source}...")
    shutil.move(source, destination)
    print(f"✅ ¡SALVADO! El dataset está seguro en: {destination}")
else:
    print("⚠️ No encuentro el archivo. ¿Seguro que se creó bien?")

⚠️ No encuentro el archivo. ¿Seguro que se creó bien?


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- SCRIPT DE RECUPERACIÓN: DIRECTO A DRIVE ---
import requests
from bs4 import BeautifulSoup
import time
import os

# RUTA SEGURA (Ajusta si tu carpeta se llama diferente)
SAFE_PATH = "/content/drive/MyDrive/ASCII-GPT/dataset_ASCII_MASSIVE_RESCATE.txt"

# ... (El mismo código de las categorías) ...
MAIN_CATEGORIES = [
    "animals", "art-and-design", "books", "buildings-and-places",
    "cartoons", "clothing-and-accessories", "comics", "computers",
    "electronics", "food-and-drinks", "holiday-and-events", "logos",
    "miscellaneous", "movies", "music", "mythology", "nature",
    "people", "plants", "religion", "space", "sports-and-outdoors",
    "television", "toys", "vehicles", "video-games", "weapons"
]

BASE_URL = "https://www.asciiart.eu"
MIN_CHARS = 20
MAX_CHARS = 2500

def get_soup(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        return BeautifulSoup(response.text, 'html.parser')
    except:
        return None

print(f"🚑 Iniciando Operación Rescate...")
print(f"💾 Los datos se guardarán en tiempo real en: {SAFE_PATH}")

# Primero recolectamos las URLs (Esto es rápido)
all_urls_to_scrape = []
print("🗺️ Mapeando sitio (esto tarda 1 min)...")
for category in MAIN_CATEGORIES:
    soup = get_soup(f"{BASE_URL}/{category}")
    if soup:
        links = soup.find_all('a', href=True)
        for link in links:
            href = link['href']
            if href.startswith(f"/{category}/") and len(href.split('/')) > 2:
                full_url = BASE_URL + href
                if full_url not in all_urls_to_scrape:
                    all_urls_to_scrape.append(full_url)
    time.sleep(0.1)

print(f"📋 Encontradas {len(all_urls_to_scrape)} subcategorías.")

# Ahora scrapeamos y guardamos al vuelo
print("🚜 Extrayendo arte...")
count = 0
with open(SAFE_PATH, "w", encoding="utf-8") as f:
    for i, url in enumerate(all_urls_to_scrape):
        soup = get_soup(url)
        if soup:
            label_parts = url.replace(BASE_URL + "/", "").split("/")
            prompt_label = " ".join([p.replace("-", " ").title() for p in label_parts])

            art_blocks = soup.find_all('pre')
            for art in art_blocks:
                text = art.get_text()
                if MIN_CHARS < len(text) < MAX_CHARS:
                    text = text.replace("\r", "")
                    entry = f"{prompt_label}\nAI:\n{text}\n<|endoftext|>\n"
                    f.write(entry)
                    count += 1

        if i % 20 == 0:
            print(f"   ✅ {count} dibujos guardados en Drive...")

print("="*50)
print(f"🎉 ¡SALVADO! Tienes {count} dibujos en tu Google Drive.")

🚑 Iniciando Operación Rescate...
💾 Los datos se guardarán en tiempo real en: /content/drive/MyDrive/ASCII-GPT/dataset_ASCII_MASSIVE_RESCATE.txt
🗺️ Mapeando sitio (esto tarda 1 min)...
📋 Encontradas 411 subcategorías.
🚜 Extrayendo arte...
   ✅ 8 dibujos guardados en Drive...
   ✅ 511 dibujos guardados en Drive...
   ✅ 817 dibujos guardados en Drive...
   ✅ 1086 dibujos guardados en Drive...
   ✅ 1284 dibujos guardados en Drive...
   ✅ 1534 dibujos guardados en Drive...
   ✅ 1715 dibujos guardados en Drive...
   ✅ 1969 dibujos guardados en Drive...
   ✅ 2209 dibujos guardados en Drive...
   ✅ 2452 dibujos guardados en Drive...
   ✅ 2577 dibujos guardados en Drive...
   ✅ 2744 dibujos guardados en Drive...
   ✅ 2916 dibujos guardados en Drive...
   ✅ 3182 dibujos guardados en Drive...
   ✅ 3411 dibujos guardados en Drive...
   ✅ 3573 dibujos guardados en Drive...
   ✅ 3936 dibujos guardados en Drive...
   ✅ 4161 dibujos guardados en Drive...
   ✅ 4382 dibujos guardados en Drive...
   ✅ 46

In [ ]:
# FUSIÓN SEGURA
outfile_path = "/content/drive/MyDrive/ASCII-GPT/dataset_FINAL_V3_READY.txt"
part1 = "/content/drive/MyDrive/ASCII-GPT/dataset_ASCII_MASSIVE_RESCATE.txt"
part2 = "/content/drive/MyDrive/ASCII-GPT/dataset_TRAVIAN_CLEAN_V2.txt"

print("⚗️ Fusionando archivos en Drive...")
with open(outfile_path, 'w') as outfile:
    # Parte 1 (Nuevo robo)
    if os.path.exists(part1):
        with open(part1) as infile:
            outfile.write(infile.read())
            outfile.write("\n<|endoftext|>\n")

    # Parte 2 (Travian antiguo)
    if os.path.exists(part2):
        with open(part2) as infile:
            outfile.write(infile.read())
            outfile.write("\n<|endoftext|>\n")

print(f"✅ ¡LISTO! Tu archivo para Kaggle es: {outfile_path}")

⚗️ Fusionando archivos en Drive...
✅ ¡LISTO! Tu archivo para Kaggle es: /content/drive/MyDrive/ASCII-GPT/dataset_FINAL_V3_READY.txt
